In [14]:
%%writefile HelperFunctions/DataWrangling/criaCorpus.py

# 1.Extração de texto dos pdf das sentenças baixas
#
# Cria Corpus (DatasetClassification.CSV) das Sentenças do TRT1
# Le os arquivos .txt e armazena-os em uma lista e depois persiste-o em disco no formato .csv.
#
import io
import pathlib
import pandas as pd
import datasets
from datasets import load_dataset
from datasets import Dataset
from os import listdir
from os.path import isfile, join
import re

    
class Corpus():
    
    def __init__(self):
        # Agrupar estas variaveis em um arquivo para carregar nas classes.
        self.FOLDER_BASE = "/home/info/MyNotebooks/Datasets/SentencasTRT1/"
        #self.PDFs_FOLDER = self.FOLDER_BASE + "PDFs/"
        self.TXTs_FOLDER = self.FOLDER_BASE + "TXTs/"
        #self.CSV_FOLDER  = self.FOLDER_BASE + "CSV/"
        self.DS_FOLDER   = self.FOLDER_BASE + "DS/"
        self.ARQ_CSV     = "Corpus_Sentencas.csv"
        self.ARQ_DS      = "Corpus_Sentencas"
        
        
    def createDS(self):
        # Obtem a lista de arquivos txt no diretorio TXTs
        #files = [self.TXTs_FOLDER+f for f in listdir(self.TXTs_FOLDER) if (isfile(join(self.TXTs_FOLDER, f)) and pathlib.Path(file).suffix == ".txt") ]
        files = [self.TXTs_FOLDER+f for f in listdir(self.TXTs_FOLDER) if isfile(join(self.TXTs_FOLDER, f))]

        dataset = load_dataset('text', data_files=files)
        dataset.save_to_disk(self.DS_FOLDER)        


Writing HelperFunctions/DataWrangling/criaCorpus.py


In [15]:
# 2. Criação do objeto
from HelperFunctions.DataWrangling.criaCorpus import Corpus
%run -i HelperFunctions/DataWrangling/criaCorpus.py
corpus = Corpus()

In [12]:
%%time
# 3. Converção dos arquivos em .PDF para texto puro .TXT
# Cria o arquivo /home/info/MyNotebooks/Datasets/SentencasTRT1/DS/dataset_dict.json e a 
# pasta Train com os arquivos: state.json, dataset_info.json e dataset.arrow
corpus.createDS()

Using custom data configuration default-42116496caae3f94
Reusing dataset text (/home/info/.cache/huggingface/datasets/text/default-42116496caae3f94/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


CPU times: user 119 ms, sys: 6.73 ms, total: 126 ms
Wall time: 765 ms


In [313]:
%%writefile HelperFunctions/DataWrangling/DataWrangling.py
# 4. DATA WRANGLING (com o dataset criado) PROPRIAMENTE:

import datasets
import re
from datasets import Dataset


class DataWrang():
    
    def __init__(self):
        # Carregamento do Dataset criado
        self.FOLDER_BASE = "/home/info/MyNotebooks/Datasets/SentencasTRT1/"
        self.DS_FOLDER   = self.FOLDER_BASE + "DS/"
        #ARQ_DS      = "Corpus_Sentencas"

        self.padraoImproc   = "improcedentes|improcedente"
        self.padraoParcProc = "procedentes em parte|procedente em parte|parcialmente procedentes|parcialmente procedente|parcialmente improcedentes|parcialmente improcedente"
        self.padraoProc     = "procedentes|procedente"
        self.padraoAcordo   = "acordo|Acordo"
        self.tam_tokeniz = 1400

        # Armazenamento em listas dos dados extraidos dos arquivos .txt para formação do novo dataset.
        self.textos         = list()
        self.dispositivos   = list()
        self.classificacoes = list()
        
        self.i = 1
        


    def extraiDispositivo(self, pattern, text):
        padrao = pattern

        texto_inicio = 0
        texto_fim    = 0

        for match in re.finditer(pattern, text.lower()):
            index = match.start()  
            value = match.group()
            print(index, value) # match only one? parece que esta encontrando outras correspondencias também.

            span = (self.tam_tokeniz - len(padrao))/2
            texto_inicio = int(match.start() - span)
            texto_fim    = int(match.start() + len(padrao) + span)
            if(texto_fim > len(text)): # nao pode ser maior que o tamanho da string
                texto_fim = len(text)

            #print(texto_inicio, texto_fim)
            #print(index)

        return(text[texto_inicio:texto_fim])

    #def preprocess_function(examples):
    #    return tokenizer(examples["text"], truncation=True)    


    def classDispositivo(self, examples):     
        
        texto = examples

        t_par    = ""
        t_imp    = ""
        t_proc   = ""
        t_outros = ""

        # parece que esta encontrando outras correspondencias também.
        t_par    = self.extraiDispositivo(self.padraoParcProc, texto)


        if len(t_par) != 0:
            dispositivo   = t_par
            classificacao = "Parcialmente procedente"
            #break
        else: 
            t_imp    = self.extraiDispositivo(self.padraoImproc, texto)
            if len(t_imp) != 0:
                dispositivo   = t_imp
                classificacao = "Improcedente"
                #break
            else: 
                t_proc   = self.extraiDispositivo(self.padraoProc, texto)
                if len(t_proc) != 0:
                    dispositivo   = t_proc
                    classificacao = "Procedente"
                    #break
                else:
                    t_outros = self.extraiDispositivo(self.padraoAcordo, texto)
                    if len(t_outros) != 0:
                        dispositivo   = t_outros
                        classificacao = "Acordo ou outros"
                    else:
                        classificacao = "Nenhuma"
                        dispositivo   = texto[self.tam_tokeniz:-1]
                    # Coloca outro if aninhando e por fim, no else residual, colocar uma flag ("none"?).
                    # Posteriormente, filtrar no dataset classificado aqueles com a flag que nao puderam
                    #  receber nenhuma das classificacoes anteriores e exclui-los do dataset anotado.

        print("PDF: ", self.i, " - Class.: ", classificacao," - Disp.: ", dispositivo)  
        self.i = self.i + 1
        self.textos.append(texto)
        self.dispositivos.append(dispositivo)
        self.classificacoes.append(classificacao)


        return(classificacao)

    
    def proccessAndSaveDsAnot(self):
        listSent  = datasets.load_from_disk(self.DS_FOLDER)['train']['text']
        #print("Tam. list. Sent: ", len(listSent), " - Tipo: ", type(listSent))
        listClass = list(map(self.classDispositivo, listSent))
        #print("Tam. list. Class: ", len(listClass), " - Tipo: ", type(listClass))

        dados = list(zip(self.textos,self.dispositivos,self.classificacoes))
        #print(len(dados))
        
        # Cria os arquivos na pasta (...)/SentencasTRT1/DsClassAnot
        df = pd.DataFrame(dados, columns=['text','disp','label'])
        dsAnot = Dataset.from_pandas(df)
        dsAnot.save_to_disk(self.FOLDER_BASE +"DsClassAnot/")



Overwriting HelperFunctions/DataWrangling/DataWrangling.py


In [314]:
import HelperFunctions.DataWrangling.DataWrangling 
%run -i HelperFunctions/DataWrangling/DataWrangling.py
dw = DataWrang()

In [315]:
# 5. Criação do objeto
# from HelperFunctions.DataWrangling.DataWrangling import DataWrang
# %run -i HelperFunctions/DataWrangling/DataWrangling.py
# dw = DataWrang()

In [316]:
dw.proccessAndSaveDsAnot()

Tam. list. Sent:  994  - Tipo:  <class 'list'>
6637 procedente
8365 procedente
8409 procedente
PDF:  1  - Class.:  Procedente  - Disp.:  go de chefe do Poder Executivo estadual assegurou a manutenção do pagamento queles que já houvessem adquirido o direito de gozar o benefício 3 Há usurpação da competência do STF inscrita no art 102 I a da CF88 quando configurado o ajuizamento de ação civil pública com o intento de dissimular o controle abstrato de constitucionalidade  de  ato  normativo  estadual  em  face  da  Constituição  Federal  4 Arquivamento da ação civil pública ante a ausência de legitimidade ativa ad causam do Parquet estadual para propor ação direta de inconstitucionalidade perante a Suprema Corte nos termos do art 103 da CF88 Precedentes 5 Reclamação julgada procedente para cassar a decisão que julgou procedente o pedido formulado nos autos da ação civil pública  declarar  a  incompetência  do  juízo  de  primeira  instância  e  determinar  o arquivamento da ação Rcl 19662

PDF:  681  - Class.:  Improcedente  - Disp.:  ende o restabelecimento da contribuição sindical obrigatória com a declaração incidental de inconstitucionalidade dos arts 578 579 582 583 587 e 602 da CLT Inicialmente verifico inexistir pretensão resistida uma vez que sequer houve a cobrança da contribuição perante o réu Dito isto o verdadeiro objeto da ação é a declaração de inconstitucionalidade dos supracitados artigos sendo vedada a sua utilização como supedâneo da Ação Direta de Inconstitucionalidade conforme uníssona jurisprudência do STF: EMENTA: AÇÃO CIVIL PÚBLICA CONTROLE INCIDENTAL DE CONSTITUCIONALIDADE QUESTÃO PREJUDICIAL POSSIBILIDADE INOCORRÊNCIA DE USURPAÇÃO DA COMPETÊNCIA DO SUPREMO TRIBUNAL FEDERAL RECLAMAÇÃO IMPROCEDENTE  O Supremo Tribunal Federal tem reconhecido a legitimidade da utilização da ação civil pública como instrumento idôneo de fiscalização incidental de constitucionalidade pela via difusa de quaisquer leis ou atos do Poder Público mesmo quando contestados e

In [ ]:
# Analise descritiva do dastaset anotado!

In [322]:
dsAnot = datasets.load_from_disk("/home/info/MyNotebooks/Datasets/SentencasTRT1/DsClassAnot/")

In [323]:
dsAnot

Dataset({
    features: ['text', 'disp', 'label'],
    num_rows: 994
})

In [372]:
dsAnot['disp'][8]

''

In [455]:
dfAnot = pd.DataFrame(dsAnot)
dfAnot = dfAnot[dfAnot.disp != '']
dfAnot.reset_index(drop=True, inplace=True)
dfAnot

In [459]:
dsAnot['disp'][8]

In [460]:
dsAnot = Dataset.from_pandas(dfAnot)
dsAnot.save_to_disk("/home/info/MyNotebooks/Datasets/SentencasTRT1/DsClassAnot/")

In [464]:
#for i in range(len(dfAnot)):
#    if dfAnot['disp'][i] == '' or dfAnot['text'][i] == '':
#        dfAnot['disp'][i] = 'na'
#dfAnot

In [466]:
dfAnot['label'].value_counts(dropna=True)

Nenhuma                    399
Improcedente               143
Procedente                 106
Acordo ou outros            47
Parcialmente procedente     26
Name: label, dtype: int64

In [468]:
total = 399+143+106+47+26
per_nenhuma = 399/total*100
print(per_nenhuma)

55.33980582524271


In [329]:
total = 672+143+106+47+26
per_nenhuma = 672/total*100
print(per_nenhuma)

67.6056338028169


In [330]:
dfAnot['label'].value_counts(dropna=True)

Nenhuma                    672
Improcedente               143
Procedente                 106
Acordo ou outros            47
Parcialmente procedente     26
Name: label, dtype: int64

In [351]:
dfNenhuma = dfAnot[dfAnot.label == "Nenhuma"].reset_index(drop=True)

In [352]:
dfNenhuma

,text,disp,label
0,Tratase de Ação Civil Pública ajuizada p...,ertinência temática entre os fins sociais da e...,Nenhuma
1,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO ...,no evidenciado transmudando a natureza e o des...,Nenhuma
2,Relatório Fundamentação Dispositivo PODER J...,x positis EXTINGUO SEM RESOLUÇÃO DO MÉRITO a p...,Nenhuma
3,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TR...,a eles concedo a gratuidade de justiça dispen...,Nenhuma
4,Vistos etc Partes ausentes A seguir fora pro...,S E DE TODOS SEUS COMPONENTES E PEÇAS M...,Nenhuma
...,...,...,...
667,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO ...,,Nenhuma
668,Relatório Fundamentação PODER JUDICIÁRIO FED...,Estadual 21979 Conforme ressalta a ré no id 1...,Nenhuma
669,Relatório Fundamentação Dispositivo Comprov...,,Nenhuma
670,PROCESSO 01003540420185010049 S E N T E N Ç ...,or veicula pela presente ação civil pública pr...,Nenhuma


In [367]:
i = 4

In [368]:
dfNenhuma['text'][i]

'  Vistos etc Partes ausentes A seguir fora proferida a seguinte SENTENÇA 1  RELATÓRIO SINDICATO  DOS  TRABALHADORES  NAS  INDÚSTRIAS  METALÚRGICAS MECÂNICAS  DE  MATERIAL  ELÉTRICO  INDÚSTRIAS  SIDERÚRGICAS  DE PROTEÇÃO TRATAMENTO TÉRMICO E TRANSFORMAÇÃO DE SUPERFÍCIES DE  APARELHOS  ELÉTRICOS  ELETRÔNICOS  ELETROELETRÔNICO  DE MATERIAIS  E  COMPONENTES  ELÉTRICO  E  ELETRÔNICOS  DE  JOGOS ELETRÔNICOS E SIMILARES INFORMÁTICA FONOGRAFIAS MULTIMÍDIA DE ARTEFATOS  DE  METAIS  NÃO  FERROSOS  METAIS  E  FERRAMENTAS  EM GERAL  DE  ARTIGOS  E  EQUIPAMENTOS  ODONTOLÓGICOS  MÉDICOS  E HOSPITALARES DE CONDUTORES ELÉTRICOS TREFILAÇÃO E LAMINAÇÃO DE METAIS NÃO FERROSOS DE ESQUADRIAS E CONSTRUÇÕES METÁLICAS DE ESTAMPARIA DE METAIS DE FORJARIA DE FUNDIÇÃO DE RETÍFICA DE FUNILARIA  DE  MÓVEIS  DE  METAL  DE  LÂMPADAS  DE  APARELHOS ELÉTRICOS DE ILUMINAÇÃO DE METAIS DE EQUIPAMENTOS FERROVIÁRIOS E RODOVIÁRIOS DE PARAFUSOS DE PORCAS REBITES E SIMILARES DE REFRIGERAÇÃO AQUECIMENTO E TRATAMENTO DE AR DE 

In [369]:
dfNenhuma['disp'][i]

'S  E  DE  TODOS  SEUS  COMPONENTES  E PEÇAS  MÁQUINAS  BALANÇAS  PESOS  E  MEDIDAS  DA  CONSTRUÇÃO NAVAL  DA  CONSTRUÇÃO  AERONÁUTICA  PEÇAS  E  EQUIPAMENTOS AERONÁUTICOS  E  AEROESPACIAL  E  SIMILARES  DAS  EMPRESAS PRINCIPAIS  TOMADORAS  DE  SERVIÇOS  DIRETA  E  INDIRETAMENTE  E TERCEIROS E SIMILARES DOS MUNICÍPIOS DE BARRA DO PIRAÍ VALENÇA MENDES  VASSOURAS  ENGENHEIRO  PAULO  DE  FRONTIN  E  PIRAÍ    RJ qualificadosaas  na  Inicial  propôs  ação  civil  pública  em  face  da  Ré DFV COMERCIAL E INDUSTRIAL LTDA Pelos fatos narrados pleiteou as parcelas descritas na inicial dando  causa o valor de R 100000 e colacionando documentos pretendendo ainda a concessão de tutela provisória É o RELATÓRIO DECIDO II  FUNDAMENTOS Por compartilhar do entendimento esposado pelo E Juiz Titular da 69 VTRJ Dr Flávio Alves Pereira em julgamento do processo n 01001941620185010069 adoto os fundamentos lá expostos em razão da identidade de matéria a ser debatida no   presente processo Da legitimidade ad

In [370]:
dfNenhuma['label'][i]

'Nenhuma'

Dataset({
    features: ['text', 'disp', 'label'],
    num_rows: 994
})

'Procedente'